In [1]:
import cv2 as cv
import numpy as np

NAME = "video_2.mp4"
VIDEO_PATH = "test-videos/"  + f"{NAME}"
OUT_PATH   = "results/" + "t1_" + f"{NAME}" 

MAX_CORNERS   = 1000
MIN_INLIERS_LK = 20


In [2]:
clahe   = cv.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
det     = cv.AKAZE_create()
matcher = cv.BFMatcher(cv.NORM_HAMMING)

def prep(gray): 
    return clahe.apply(gray)

def reseed_klt(gray, corners):
    mask = np.zeros(gray.shape[:2], np.uint8)
    cv.fillPoly(mask, [corners.reshape(-1,2).astype(np.int32)], 255)
    return cv.goodFeaturesToTrack(gray, MAX_CORNERS, 0.01, 7, blockSize=7, mask=mask)

def redetect(gray):
    kp, des = det.detectAndCompute(prep(gray), None)
    if des is None or des0 is None:
        return None

    good = [m for m, n in matcher.knnMatch(des0, des, k=2) ]

    src = np.float32([kp0[m.queryIdx].pt for m in good]).reshape(-1,1,2)
    dst = np.float32([kp[m.trainIdx].pt  for m in good]).reshape(-1,1,2)

    Hm, _ = cv.findHomography(src, dst, cv.RANSAC,)

    return Hm

In [3]:
cap = cv.VideoCapture(VIDEO_PATH)
ok, frame0 = cap.read()

H, W = frame0.shape[:2]
fps = cap.get(cv.CAP_PROP_FPS) or 30
writer = cv.VideoWriter(OUT_PATH, cv.VideoWriter_fourcc(*"mp4v"), fps, (W, H))

gray0 = cv.cvtColor(frame0, cv.COLOR_BGR2GRAY)
kp0, des0 = det.detectAndCompute(prep(gray0), None)

corners0 = np.float32([[0,0],[W-1,0],[W-1,H-1],[0,H-1]]).reshape(-1,1,2)
corners  = corners0.copy()

prev_gray = gray0
p_prev = reseed_klt(prev_gray, corners)
lost = False

In [4]:
while True:
    ok, frame = cap.read()
    if not ok:
        break

    gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)

    if not lost:

        if p_prev is None or len(p_prev) < 8:
            lost = True
        else:
            p_next, st, _  = cv.calcOpticalFlowPyrLK(prev_gray, gray, p_prev, None, )
            p_back, st2, _ = cv.calcOpticalFlowPyrLK(gray, prev_gray, p_next, None, )

            fb = np.linalg.norm(p_prev - p_back, axis=2).ravel()
            good = (st.ravel() == 1) & (st2.ravel() == 1) 

            p0g = p_prev[good]
            p1g = p_next[good]

            if len(p0g) >= 8:
                Hm, inl = cv.findHomography(p0g, p1g, cv.RANSAC, 3.0)
                if Hm is not None and inl is not None and int(inl.sum()) >= MIN_INLIERS_LK:
                    inl = inl.ravel().astype(bool)
                    corners = cv.perspectiveTransform(corners, Hm)
                    p_prev = p1g[inl].reshape(-1,1,2)
                    prev_gray = gray
                else:
                    lost = True
            else:
                lost = True

    if lost:
        Habs = redetect(gray)
        if Habs is not None:
            corners = cv.perspectiveTransform(corners0, Habs)
            p_prev = reseed_klt(gray, corners)
            prev_gray = gray
            lost = False

    cv.polylines(frame, [corners.reshape(-1,2).astype(np.int32)], True, (0,255,0) if not lost else (0,0,255), 3)
    writer.write(frame)

cap.release()
writer.release()
print("Saved:", OUT_PATH)

Saved: results/t1_video_2.mp4
